In [1]:
pip install requests pandas tqdm wordcloud

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import time
import sys
import concurrent.futures
from tqdm import tqdm
import csv

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
HEADERS = {"Accept": "application/json"}
ANOS_ANALISE = [2023, 2024, 2025]

MAX_WORKERS = 20

def coletar_dados_paginados(url_inicial, log_prefix):

    todos_os_dados = []
    url_pagina_atual = url_inicial
    pagina_num = 1
    
    while url_pagina_atual:
        try:
            response = requests.get(url_pagina_atual, headers=HEADERS, timeout=30)
            
            if response.status_code == 200:
                dados = response.json()
                todos_os_dados.extend(dados['dados'])

                links = dados.get('links', [])
                url_pagina_atual = None
                for link in links:
                    if link.get('rel') == 'next':
                        url_pagina_atual = link.get('href')
                        break
                        
            elif response.status_code == 429:
                print(f"{log_prefix} - ! RATE LIMIT (429) ! Pausando por 5s e tentando novamente...")
                time.sleep(5)
                
            else:
                print(f"{log_prefix} - Erro na página {pagina_num} ({url_pagina_atual}): {response.status_code}")
                url_pagina_atual = None
                
        except requests.exceptions.RequestException as e:
            print(f"{log_prefix} - Erro de conexão: {e}")
            url_pagina_atual = None
        
        pagina_num += 1
    
    if pagina_num > 1 and len(todos_os_dados) > 0:
         print(f"{log_prefix} - Coleta finalizada. {len(todos_os_dados)} registros obtidos.")
    elif len(todos_os_dados) == 0:
        print(f"{log_prefix} - Coleta finalizada. Nenhum registro encontrado.")

    return todos_os_dados

def coletar_deputados():
    print("\n[COLETANDO DEPUTADOS] - (Sequencial)")
    url = f"{BASE_URL}/deputados?ordem=ASC&ordenarPor=nome&itens=100"
    dados = coletar_dados_paginados(url, "[Deputados]")
    return pd.DataFrame(dados)

def fetch_despesas_deputado_worker(task_args):
    deputado_id, ano, i, total = task_args
    log_prefix = f"  [Despesa {ano} - Dep. {i}/{total} (ID: {deputado_id})]"
    
    url = f"{BASE_URL}/deputados/{deputado_id}/despesas?ano={ano}&ordem=ASC&ordenarPor=dataDocumento&itens=100"
    
    dados_despesa = coletar_dados_paginados(url, log_prefix)
    
    for despesa in dados_despesa:
        despesa['idDeputado'] = deputado_id
        
    return dados_despesa

def coletar_despesas(lista_ids_deputados, ano):
    master_lista_despesas = []
    total_deputados = len(lista_ids_deputados)
    
    tasks = [(dep_id, ano, i+1, total_deputados) for i, dep_id in enumerate(lista_ids_deputados)]

    print(f"\n[COLETANDO DESPESAS {ano}] - (Paralelo com {MAX_WORKERS} workers)")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results_list_of_lists = list(
            tqdm(
                executor.map(fetch_despesas_deputado_worker, tasks), 
                total=total_deputados, 
                desc=f"Coletando Despesas {ano}"
            )
        )

    for sublist in results_list_of_lists:
        master_lista_despesas.extend(sublist)
        
    print(f"\n--- Coleta de Despesas para {ano} Finalizada ---")
    return pd.DataFrame(master_lista_despesas)

def coletar_proposicoes(ano):
    print(f"\n[COLETANDO PROPOSIÇÕES {ano}] - (Sequencial)")
    url = f"{BASE_URL}/proposicoes?ano={ano}&ordem=ASC&ordenarPor=id&itens=100"
    log_prefix = f"[Proposições {ano}]"
    dados = coletar_dados_paginados(url, log_prefix)
    return pd.DataFrame(dados)

def main():
    
    df_deputados = coletar_deputados()
    
    if df_deputados.empty:
        print("Erro: Nenhum deputado coletado. Abortando.")
        sys.exit()
        
    df_deputados.to_csv("deputados.csv", index=False, encoding='utf-8-sig')
    print("\n[ETAPA 1/3] SUCESSO! 'deputados.csv' salvo.")
    
    ids_deputados = df_deputados['id'].tolist()
    
    for ano in ANOS_ANALISE:
        print(f"\n===== PROCESSANDO ANO: {ano} =====")
        
        df_despesas = coletar_despesas(ids_deputados, ano)
        
        if not df_despesas.empty:
            nome_arquivo = f"despesas_{ano}.csv"
            df_despesas.to_csv(nome_arquivo, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_ALL)
            print(f"\n[ETAPA 2/3 - Ano {ano}] SUCESSO! '{nome_arquivo}' salvo (AGORA COM A COLUNA 'idDeputado').")
        else:
            print(f"\n[ETAPA 2/3 - Ano {ano}] AVISO: Nenhuma despesa encontrada ou coletada.")

        df_proposicoes = coletar_proposicoes(ano)
        
        if not df_proposicoes.empty:
            nome_arquivo_prop = f"proposicoes_{ano}.csv"
            
            df_proposicoes.to_csv(nome_arquivo_prop, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_ALL)
            
            print(f"\n[ETAPA 3/3 - Ano {ano}] SUCESSO! '{nome_arquivo_prop}' salvo.")
        else:
            print(f"\n[ETAPA 3/3 - Ano {ano}] AVISO: Nenhuma proposição encontrada ou coletada.")
            
    print("\n--- COLETA FINALIZADA ---")

if __name__ == "__main__":
    main()


[COLETANDO DEPUTADOS] - (Sequencial)
[Deputados] - Coleta finalizada. 513 registros obtidos.

[ETAPA 1/3] SUCESSO! 'deputados.csv' salvo.

===== PROCESSANDO ANO: 2023 =====

[COLETANDO DESPESAS 2023] - (Paralelo com 20 workers)


Coletando Despesas 2023:   0%|                          | 0/513 [00:00<?, ?it/s]

  [Despesa 2023 - Dep. 12/513 (ID: 73579)] - Coleta finalizada. 72 registros obtidos.
  [Despesa 2023 - Dep. 11/513 (ID: 204549)] - Coleta finalizada. 176 registros obtidos.
  [Despesa 2023 - Dep. 18/513 (ID: 220542)] - Coleta finalizada. 273 registros obtidos.
  [Despesa 2023 - Dep. 20/513 (ID: 221148)] - Coleta finalizada. 247 registros obtidos.
  [Despesa 2023 - Dep. 3/513 (ID: 204560)] - Coleta finalizada. 202 registros obtidos.


Coletando Despesas 2023:   0%|                  | 1/513 [00:03<32:30,  3.81s/it]

  [Despesa 2023 - Dep. 16/513 (ID: 178972)] - Coleta finalizada. 243 registros obtidos.
  [Despesa 2023 - Dep. 4/513 (ID: 204528)] - Coleta finalizada. 310 registros obtidos.
  [Despesa 2023 - Dep. 1/513 (ID: 204379)] - Coleta finalizada. 243 registros obtidos.
  [Despesa 2023 - Dep. 2/513 (ID: 220714)] - Coleta finalizada. 226 registros obtidos.
  [Despesa 2023 - Dep. 17/513 (ID: 204571)] - Coleta finalizada. 219 registros obtidos.
  [Despesa 2023 - Dep. 13/513 (ID: 220538)] - Coleta finalizada. 266 registros obtidos.
  [Despesa 2023 - Dep. 29/513 (ID: 220715)] - Coleta finalizada. 30 registros obtidos.
  [Despesa 2023 - Dep. 15/513 (ID: 204501)] - Coleta finalizada. 302 registros obtidos.
  [Despesa 2023 - Dep. 6/513 (ID: 74646)] - Coleta finalizada. 383 registros obtidos.


...

Coletando Despesas 2025:  94%|███████████████ | 483/513 [02:22<00:08,  3.61it/s]

  [Despesa 2025 - Dep. 494/513 (ID: 141555)] - Coleta finalizada. 296 registros obtidos.
  [Despesa 2025 - Dep. 483/513 (ID: 220597)] - Coleta finalizada. 456 registros obtidos.
  [Despesa 2025 - Dep. 513/513 (ID: 220552)] - Coleta finalizada. 478 registros obtidos.
  [Despesa 2025 - Dep. 507/513 (ID: 204559)] - Coleta finalizada. 415 registros obtidos.
  [Despesa 2025 - Dep. 511/513 (ID: 160592)] - Coleta finalizada. 566 registros obtidos.
  [Despesa 2025 - Dep. 509/513 (ID: 220558)] - Coleta finalizada. 433 registros obtidos.
  [Despesa 2025 - Dep. 510/513 (ID: 204517)] - Coleta finalizada. 310 registros obtidos.


Coletando Despesas 2025: 100%|████████████████| 513/513 [02:33<00:00,  3.34it/s]

  [Despesa 2025 - Dep. 496/513 (ID: 178992)] - Coleta finalizada. 716 registros obtidos.

--- Coleta de Despesas para 2025 Finalizada ---



[ETAPA 2/3 - Ano 2025] SUCESSO! 'despesas_2025.csv' salvo (AGORA COM A COLUNA 'idDeputado').

[COLETANDO PROPOSIÇÕES 2025] - (Sequencial)
[Proposições 2025] - Coleta finalizada. 42702 registros obtidos.

[ETAPA 3/3 - Ano 2025] SUCESSO! 'proposicoes_2025.csv' salvo.

--- COLETA FINALIZADA ---


In [1]:
import requests
import pandas as pd
import time
import concurrent.futures
from tqdm import tqdm
import json
import os

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"
HEADERS = {"Accept": "application/json"}
MAX_WORKERS = 10

def coletar_dados_endpoint_unico(url, log_prefix):
    try:
        response = requests.get(url, headers=HEADERS, timeout=30)
        
        if response.status_code == 200:
            return response.json()['dados']
        
        elif response.status_code == 429:
            print(f"{log_prefix} - ! RATE LIMIT (429) ! Pausando por 10s...")
            time.sleep(10)
            return coletar_dados_endpoint_unico(url, log_prefix) # Tenta novamente
            
        elif response.status_code == 404:
            print(f"{log_prefix} - ! NÃO ENCONTRADO (404) ! URL: {url}")
            return None # Não encontrado
            
        else:
            print(f"{log_prefix} - Erro {response.status_code} na URL: {url}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"{log_prefix} - Erro de conexão: {e}. Pausando por 5s.")
        time.sleep(5)
        return coletar_dados_endpoint_unico(url, log_prefix) # Tenta novamente

def fetch_detalhes_proposicao_worker(task_args):
    prop_id, i, total = task_args
    log_prefix = f"[Prop. {i}/{total} (ID: {prop_id})]"

    # 1. Coletar Detalhes
    url_detalhe = f"{BASE_URL}/proposicoes/{prop_id}"
    detalhes = coletar_dados_endpoint_unico(url_detalhe, log_prefix + " [Detalhe]")
    
    # 2. Coletar Autores
    url_autores = f"{BASE_URL}/proposicoes/{prop_id}/autores"
    autores = coletar_dados_endpoint_unico(url_autores, log_prefix + " [Autores]")

    # 3. Coletar Tramitações
    url_tram = f"{BASE_URL}/proposicoes/{prop_id}/tramitacoes"
    tramitacoes = coletar_dados_endpoint_unico(url_tram, log_prefix + " [Tramitações]")

    # Retorna tudo para o ID da proposição
    return {
        "id_proposicao": prop_id,
        "detalhes": detalhes,
        "autores": autores,
        "tramitacoes": tramitacoes
    }

def main_fase2_proposicoes(ano):
    
    arquivo_csv_base = f"proposicoes_{ano}.csv"
    arquivo_json_saida = f"proposicoes_detalhadas_{ano}.json"
    
    print(f"\n===== INICIANDO FASE 2: DETALHAMENTO PROPOSIÇÕES {ano} =====")
    
    try:
        df_base = pd.read_csv(arquivo_csv_base)
    except FileNotFoundError:
        print(f"Erro: Arquivo base '{arquivo_csv_base}' não encontrado.")
        print("Rode o script principal primeiro.")
        return

    ids_proposicoes = df_base['id'].unique().tolist()
    total_props = len(ids_proposicoes)
    
    if total_props == 0:
        print("Nenhum ID de proposição encontrado no arquivo.")
        return

    tasks = [(prop_id, i+1, total_props) for i, prop_id in enumerate(ids_proposicoes)]
    
    todos_os_detalhes = []

    print(f"Iniciando coleta de detalhes para {total_props} proposições (Workers: {MAX_WORKERS})")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(
            tqdm(
                executor.map(fetch_detalhes_proposicao_worker, tasks),
                total=total_props,
                desc=f"Detalhando Proposições {ano}"
            )
        )
    
    with open(arquivo_json_saida, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
        
    print(f"\n[FASE 2 - Ano {ano}] SUCESSO! '{arquivo_json_saida}' salvo.")


if __name__ == "__main__":
    ANOS_ANALISE = [2023, 2024, 2025]
    for ano in ANOS_ANALISE:
        if os.path.exists(f"proposicoes_{ano}.csv"):
            main_fase2_proposicoes(ano)
        else:
            print(f"Aviso: Arquivo proposicoes_{ano}.csv não existe. Pulando.")
            
    print("\n--- DETALHAMENTO FINALIZADO ---")


===== INICIANDO FASE 2: DETALHAMENTO PROPOSIÇÕES 2023 =====
Iniciando coleta de detalhes para 28144 proposições (Workers: 10)


Detalhando Proposições 2023:   0%|         | 1/28144 [00:03<28:07:15,  3.60s/it]

[Prop. 13/28144 (ID: 2345496)] [Autores] - ! RATE LIMIT (429) ! Pausando por 10s...


Detalhando Proposições 2023:   0%|         | 2/28144 [00:04<17:03:28,  2.18s/it]

[Prop. 11/28144 (ID: 2345494)] [Tramitações] - ! RATE LIMIT (429) ! Pausando por 10s...
[Prop. 24/28144 (ID: 2345583)] [Detalhe] - ! RATE LIMIT (429) ! Pausando por 10s...
[Prop. 23/28144 (ID: 2345582)] [Tramitações] - ! RATE LIMIT (429) ! Pausando por 10s...
[Prop. 13/28144 (ID: 2345496)] [Autores] - ! RATE LIMIT (429) ! Pausando por 10s...
[Prop. 11/28144 (ID: 2345494)] [Tramitações] - ! RATE LIMIT (429) ! Pausando por 10s...
[Prop. 47/28144 (ID: 2345609)] [Tramitações] - ! RATE LIMIT (429) ! Pausando por 10s...
[Prop. 13/28144 (ID: 2345496)] [Autores] - ! RATE LIMIT (429) ! Pausando por 10s...


...

[Prop. 42592/42702 (ID: 2586461)] [Detalhe] - ! RATE LIMIT (429) ! Pausando por 10s...
[Prop. 42593/42702 (ID: 2586462)] [Detalhe] - ! RATE LIMIT (429) ! Pausando por 10s...
[Prop. 42590/42702 (ID: 2586459)] [Autores] - ! RATE LIMIT (429) ! Pausando por 10s...


Detalhando Proposições 2025: 100%|█████▉| 42636/42702 [4:59:53<00:18,  3.65it/s]

[Prop. 42649/42702 (ID: 2586589)] [Detalhe] - ! RATE LIMIT (429) ! Pausando por 10s...


Detalhando Proposições 2025: 100%|██████| 42702/42702 [5:00:18<00:00,  2.37it/s]



[FASE 2 - Ano 2025] SUCESSO! 'proposicoes_detalhadas_2025.json' salvo.

--- DETALHAMENTO FINALIZADO ---
